In [1]:
from pyspark import SparkContext, SparkConf
import os
import shutil
import string
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jamesnguyen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
conf = SparkConf().setAppName("WordCount)").setMaster('local')
sc = SparkContext(conf = conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/28 14:54:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc

<SparkContext master=local appName=WordCount)>

In [4]:
books = [
    './book1.txt',
    './book2.txt',
    './book3.txt',
]

def get_content(path):
    with open(path, 'r') as file:
        return file.read()

bookdata = []
for path in books:
    bookdata.append(get_content(path))

books_rdd = sc.parallelize(bookdata)


In [5]:
print(type(books_rdd))
print('num partitions ', books_rdd.getNumPartitions())
print('first element len = ', len(books_rdd.first()))
print('elements count = ', books_rdd.count())

<class 'pyspark.rdd.RDD'>
num partitions  1


23/11/28 14:54:20 WARN TaskSetManager: Stage 0 contains a task of very large size (1257 KiB). The maximum recommended task size is 1000 KiB.


first element len =  81255
elements count =  3


23/11/28 14:54:21 WARN TaskSetManager: Stage 1 contains a task of very large size (1257 KiB). The maximum recommended task size is 1000 KiB.


In [42]:
input = './sample.txt'
output = './sample_output_d'
# text_rdd = sc.textFile('./sample.txt')

### Basic wordcount

In [114]:
output = 'basic_out'
try:
    shutil.rmtree(output)
except Exception as e:
    print(e)

basic = books_rdd.flatMap(lambda x: x.split())\
                    .map(lambda word: (word, 1))\
                    .reduceByKey(lambda a,b: a + b)\
                    # .sortBy(lambda x: x[1], ascending=False)\
# print(typefinal)
basic.saveAsTextFile(output)

23/11/28 14:31:32 WARN TaskSetManager: Stage 104 contains a task of very large size (1257 KiB). The maximum recommended task size is 1000 KiB.


### Extend WordCount

1. case insensitive ( to lower case)
2. ignore punctuation
3. ignore stop words
4. output sorted in descending order by value

### Ignore punctuation

In [122]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [124]:

def remove_punctuation(x):
    return re.sub(r'[^a-zA-Z]', ' ', x)


In [126]:
a = 'passages--the'
b = remove_punctuation(a)
print(b.split())

['passages', 'the']


In [8]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
output = 'extended_output'
try:
    shutil.rmtree(output)
except Exception as e:
    print(e)

def is_stopword(x):
    return x in stopwords.words('english')

def process(x):
    #lower case x
    x = x.lower()
    #remove punctuation
    x = re.sub(r'[^a-zA-Z]', ' ', x)
    return x.split()

    
basic = books_rdd.flatMap(lambda x: process(x))\
                    .filter(lambda x: is_stopword(x) == False)\
                    .map(lambda word: (word, 1))\
                    .reduceByKey(lambda a,b: a + b)\
                    .sortBy(lambda x: x[1], ascending=False)\
# print(typefinal)
basic.saveAsTextFile(output)

23/11/28 14:55:10 WARN TaskSetManager: Stage 2 contains a task of very large size (1257 KiB). The maximum recommended task size is 1000 KiB.


#Remove 